#Instalamos pytorch


In [1]:
#pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

#Clonamos el repositorio para obtener el dataset

In [2]:
!git clone https://github.com/joanby/deeplearning-az.git

Cloning into 'deeplearning-az'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 10153 (delta 25), reused 39 (delta 16), pack-reused 10096
Receiving objects: 100% (10153/10153), 236.95 MiB | 36.84 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Checking out files: 100% (10108/10108), done.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Importar las librerías

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importar el dataset


In [3]:
movies = pd.read_csv("ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users  = pd.read_csv("ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparar el conjunto de entrenamiento y elconjunto de testing

In [4]:
training_set = pd.read_csv("ml-100k/u1.base", sep = "\t", header = None)
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("ml-100k/u1.test", sep = "\t", header = None)
test_set = np.array(test_set, dtype = "int")

# Obtener el número de usuarios y de películas

In [5]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

# Convertir los datos en un array X[u,i] con usuarios u en fila y películas i en columna


In [6]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [7]:
training_set = convert(training_set)
test_set = convert(test_set)

# Convertir los datos a tensores de Torch

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Crear la arquitectura de la Red Neuronal

In [9]:
# Creamos una clase del Stacked AE (para luego crear instancias de objetos de la misma)
# En Pytorch no viene preparada una clase para el AE, pero si nos permite generar herencia (generamos clase SAE, que extrae características primarias de la clase Module de Pytorch)
# Usamos SAE para codificar las entradas en varias capas ocultas (también solucionaremos el problema de autocompletación de las mismas)
class SAE(nn.Module): # añadimos la clase de la cual heredamos (ahora AE tiene las variables, métodos y funciones de Module)
    def __init__(self, ): # constructor (define los parámetros que inicializarán los objetos) (podemos agg también variables propias, pero usamos las de Module)
        super(SAE, self).__init__() # llamamos al constructor del 'padre' para inicializar sus parámetros y funciones (super)
        self.fc1 = nn.Linear(nb_movies, 20) # primera capa oculta ligada al SAE, creada con Linear (capa Full Conection de Module), asignándole el número de entradas brindadas (todas las valoraciones del usuario en cuestión, columnas) y el número de nodos de capas ocultas (probar con GridSeaarch, 20). Cada neurona en AE representa la extracción de una característica de todas las entradas (películas), ya sean por actores, genero, etc 
        self.fc2 = nn.Linear(20, 10) # segunda capa oculta, asignándole la salida de la capa anterior (20) y el número de nodos para la capa intermedia (10). Pasamos a un número menor de características en base a la capa anterior
        self.fc3 = nn.Linear(10, 20) # tercera capa oculta, que inicie el proceso de descodificación para intentar reconstruir las entradas originales (por eso el número de nodos aumenta con respecto a la anterior, de manera simétrica)
        self.fc4 = nn.Linear(20, nb_movies) # cuarta capa oculta, que tendrá la misma cantidad de nodos que los proporcionados originalmente, para poder ver el rendimiento del proceso de codificación en la reconstrucción, reconocimiento de características y resumen de las entradas originales (aumenta de nodos en relación al número de columnas inicial)
        self.activation = nn.Sigmoid() # f de activación de Sigmoid para activar las neuronas cuando la valoración entre en ellas (Sigmoide, Tanh o Relu) (Sigmoide potencia los extremos de activación o desactivación de la neurona)
        # Buscamos explicitar el proceso de Condificación y Descodificación con la siguiente función, que aplica las funciones de activación en cada capa oculta, que terminará por devolver las calificaciones pronosticadas, que se compararán con las originalmente proporcionadas
    def forward(self, x): # información fluyendo hacia adelante (Forward), siendo x el vector de entrada de datos (valoraciones de todos los usuarios)
        x = self.activation(self.fc1(x)) # codificamos el vector de entrada (x) con la primera capa oculta (pasa de 1683 a 20), aplicando Sigmoide a los resultados de esa capa oculta (a esa extracción)
        x = self.activation(self.fc2(x)) # codificamos la salida de la primera capa con la segunda capa oculta (pasa de 20 a 10), aplicando Sigmoide a esa extracción
        x = self.activation(self.fc3(x)) # descodificamos la salida de la segunda capa con la tercera capa oculta (pasa de 10 a 20), aplicamos Sigmoide también a esa reconstrucción
        x = self.fc4(x) # finalizamos la descodificación de los datos con la cuarta capa oculta, brindando el mismo número de salidas que las ingresadas a la red (pasa de 20 a 1683 características, buscando ser similar a las entradas)
        # Pero en esta última solo quiero realizar una predicción final, por lo que no busco aplicar función de activación para despertar o no neuronas. Si aplico Siogmoid me mantiene los valores entre 0 y 1, no siendo posible de comparar las predicciones con las entradas originales
        return x # devuelvo el vector de las valoraciones pronosticadas por la Condificación y Descodificación de la SAE

In [10]:
sae = SAE() # creamos un objeto del Stacked AutoEncoder
criterion = nn.MSELoss() # usamos como función de pérdida el MSE (error cuadrado medio)
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) # usamos como optimizador el RMSprop (probar Adam, GD, etc) 
# Aclaramos los argumentos de learning_rate (cuanto más pequeños son, más tardan en converger pero más precisos son los resultados) y weight_decay (bajada en el peso original del lr para regular convergencia, osea que el lr se vaya adaptando a la convergencia, como mucho irá bajando hasta la mitad de su valor original) con la función parameters de Module incorporada en el objeto sae

# Entrenar el SAE

In [11]:
nb_epoch = 200 # aclaramos el número de épocas del entrenamiento con SAE (num de veces que se actualizarán los pesos, ya sea ind o en bloques, por cada vuelta al dataset)
for epoch in range(1, nb_epoch+1): # bucle que recorre las épocas del 1 al 200 (del 1 al 200)
    train_loss = 0 # inicializamos la función de pérdida del entrenamiento (la reiniciamos a 0 en cada época, ya que es independiente en cada una de ellas)
    s = 0. # inicializamos el contador de usuarios que al menos emitieron una valoración (prescindimos de los usuarios sin valoraciones). Lo usaremos luego para calcular el RMSE relativo
    for id_user in range(nb_users): # bucle que recorre los usuarios (filas) dentro de cada época (rango de usuarios de los cuales tenemos las valoraciones). Como recorremos de 1 en 1, aplicamos Reinforcement Learning (podríamos probar recorrerlos en bloques como en RBM)
        input = Variable(training_set[id_user]).unsqueeze(0) # obtenemos como entrada a la SAE de las valoraciones de ese usuario/fila (agregamos una dimensión adicional falsa que simule el lote/bloque de obs, para que me lo acepte Pytorch, con la función de Variable unsqueeze(0), para añadir la dimensión en la posición 0). Así, luego de cada usuario se corregiran los pesos
        target = input.clone() # copiamos el vector de entrada para luego al final del entrenamiento, tomarlo como objetivo a predecir 
        if torch.sum(target.data > 0) > 0: # if para optimizar la memoria, entrenando sólo con usuarios que han calificado al menos 1 película (ya que los que no, no tendrían entradas para reconstruir). Sumo el vector de valoraciones positivas del usuario, y si es positiva, al menos ha valorado 1 película (recordar que el 0 indicaba que no la había visto) (podriamos ser más finos y poner al final >10 para entrenar solo los usuarios que al menos han valorado 10 películas)
            output = sae.forward(input) # guardamos la salida de la SAE en output, a la que le proporcionaremos los datos de entrada para que Codifique y Descodifique (con la función forward del objeto sae). Esta sería la predicción de SAE de las valoraciones originales
            target.require_grad = False # en el target (objetivo), no requerimos de ningún cálculo del GD, por lo que lo anulamos (reduce el número de operaciones, al solo calcularlo sobre input)
            output[target == 0] = 0 # establecemos que la predicción de las películas no vistas se mantenga en 0 (excluimos del cálculo del GD y del error las películas no vistas por el usuario)
            loss = criterion(output, target) # calculamos el error de pérdida utilizando el criterio de pérdida anteriormente creado y comparando los 2 sets (los pronósticos del AE, output; con las verdaderas entradas, target)
            # La media no es sobre todas las películas, sino sobre las que realmente ha valorado (ratio de películas valoradas). Esto lo hacemos para medir/repartir el error respecto a las películas valoradas por el usuario, no sobre el total (que incluye las no vistas)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) # factor/media de corrección (num de películas / num de valoraciones vistas por el usuario, que no sean 0). Lo elevamos a la 10 elev a la -10 para no truncar la corrección por dividir por 0
            loss.backward() # llamamos al método de propagación hacia atras para corregir los pesos que anteriormente declaramos
            train_loss += np.sqrt(loss.data*mean_corrector) # Para RMSE calculamos la raiz de la multiplicación del cáclulo del MSE (es la suma de errores / n_pelis_valoradas)
            # SI QUISIERAMOS EL MSE HACER: train_loss += loss.data[0]*mean_corrector (calculamos el MSE con respecto al valor original de train_loss (0). Tomamos la media del error con respecto al número de películas vistas por el usuario)
            s += 1. # actualizamos el contador de usuarios con al menos 1 película vista
            optimizer.step() # usamos el optimizador para actualizar los pesos, haciendo un paso hacia adelante en la dirección emitida por la función de pérdida
    print("Epoch: "+str(epoch)+", Loss: "+str(train_loss/s)) # vemos el RMSE acumulado por época dividido por el número de usuarios que han valorado al menos 1 película para obtener el promedio hasta esa época

# El paso de la función de pérdida (propagación hacia atras) decide la dirección en la que se deben actualizar los pesos para minimizar el error, mientras que el paso de optimización decide la intensidad del cambio (por cuanto hay que multiplicar los pesos en esa dirección, utilizando el lr y decay)
# La pérdida nos inidicará en este caso alrededor de que valores en el contexto del problema se equivocará prediciendo el modelo (si es 1, dirá que en promedio se equivoca prediciendo en 1 punto de valoración, ya sea más o menos)
# Podemos subir el ratio de lr y dacay para que converja más rápido, pero con la configuración actual converje en torno al 0,91
# Luego lo tengo que comparar con el conjunto de test (viendo si estamos ante Overffiting o Underffiting)

Epoch: 1, Loss: tensor(1.7710)
Epoch: 2, Loss: tensor(1.0969)
Epoch: 3, Loss: tensor(1.0534)
Epoch: 4, Loss: tensor(1.0383)
Epoch: 5, Loss: tensor(1.0308)
Epoch: 6, Loss: tensor(1.0268)
Epoch: 7, Loss: tensor(1.0239)
Epoch: 8, Loss: tensor(1.0221)
Epoch: 9, Loss: tensor(1.0208)
Epoch: 10, Loss: tensor(1.0198)
Epoch: 11, Loss: tensor(1.0189)
Epoch: 12, Loss: tensor(1.0183)
Epoch: 13, Loss: tensor(1.0179)
Epoch: 14, Loss: tensor(1.0175)
Epoch: 15, Loss: tensor(1.0174)
Epoch: 16, Loss: tensor(1.0171)
Epoch: 17, Loss: tensor(1.0167)
Epoch: 18, Loss: tensor(1.0166)
Epoch: 19, Loss: tensor(1.0165)
Epoch: 20, Loss: tensor(1.0161)
Epoch: 21, Loss: tensor(1.0161)
Epoch: 22, Loss: tensor(1.0161)
Epoch: 23, Loss: tensor(1.0159)
Epoch: 24, Loss: tensor(1.0159)
Epoch: 25, Loss: tensor(1.0157)
Epoch: 26, Loss: tensor(1.0156)
Epoch: 27, Loss: tensor(1.0153)
Epoch: 28, Loss: tensor(1.0152)
Epoch: 29, Loss: tensor(1.0130)
Epoch: 30, Loss: tensor(1.0110)
Epoch: 31, Loss: tensor(1.0102)
Epoch: 32, Loss: 

# Evaluar el conjunto de test en nuestro SAE

In [12]:
# Para predecir sobre el conjunto de test, utilizamos una estructura similar al entrenamiento, solo que con algunas diferencias
# Ya no necesitamos el bucle de las épocas, ya que la NN está entrenada (necesito sólo una época general que mida el resultado sobre los usuarios que esten en test)
# Ya no necesitamos la propagación hacia atras del error ni el optimizador, ya que no estamos en la fase de entrenamiento
test_loss = 0
s = 0.
for id_user in range(nb_users):
    # Compararemos las calificaciones reales del conjunto de test con las calificaciones predichas a partir del conjunto de entrenamiento (cuantificando el error con las valoraciones efectivamente hechas por tal usuario, ya que con las que no vió no tengo manera de comparar)
    input = Variable(training_set[id_user]).unsqueeze(0) # mantenemos el conjunto de entrenamiento como input, ya que tomamos las valoraciones conocidas del usuario para predecir sobre las que conocemos y las que no
    target = Variable(test_set[id_user]).unsqueeze(0) # tomaremos como objetivo a predecir las valoraciones de los usuarios de test
    if torch.sum(target.data > 0) > 0: # condición para predecir: que al menos haya valorado una película
        output = sae.forward(input) # en base a las calificaciones del usuario en el pasado (input), predecimos sobre la valoración futura 
        target.require_grad = False # el conj objetivo no cambia
        output[target == 0] = 0 # necesito mantener en 0 la predicción de las películas no vistas del set de test para medir el error, pero en el futuro quiero saber predecir los valores de 0
        loss = criterion(output, target) # mido la pérdida sobre las películas vistas
        mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) # media correctora sobre las películas valoradas por el usuario
        test_loss += np.sqrt(loss.data*mean_corrector) # acumulamos el RMSE del error, corregido con la media correctora para que sea sobre las películas vistas en vez de sobre todas
        s += 1. # actualizamos el contador de usuarios con al menos 1 película vista

In [15]:
print("Test Loss: "+str(test_loss/s)) # ya no hay épocas, podemos ver directamente en cuánto se equivoca el algoritmo respecto a los usuarios de test

Test Loss: tensor(0.9475)


# Predecimos sobre todas las valoraciones del usuario (películas vistas y no vistas)

In [18]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0) # mantenemos el conjunto de entrenamiento como input
    target = Variable(test_set[id_user]).unsqueeze(0) # tomamos como objetivo las valoraciones del test set
    if torch.sum(target.data > 0) > 0: # condición para predecir: que haya al menos una película valorada
        output = sae.forward(input) # predicción sobre las películas vistas y no vistas
        target.require_grad = False
        # No excluimos las predicciones de las películas no vistas
        loss = criterion(output, target) # medimos la pérdida solo sobre las películas vistas
        mean_corrector = nb_movies/float(torch.sum(target.data > 0)+1e-10) # media correctora sobre las películas valoradas
        test_loss += np.sqrt(loss.data * mean_corrector)
        s += 1.
        
        # Filtramos las predicciones sobre las películas no vistas
        unseen_predictions = output[target.data == 0].data.numpy() # extraemos las predicciones para las películas no vistas (donde target es 0)
        unseen_movies = np.where(target.data == 0)[1] # obtenemos los índices de las películas no vistas
        print(f"Usuario {id_user + 1}: Películas no vistas - {unseen_movies}, Predicciones - {unseen_predictions}")

# Podremos recomendarle a los usuarios las películas que posean una mayor valoración predicha con una mayor probabilidad de que le guste

Usuario 1: Películas no vistas - [   0    1    2 ... 1679 1680 1681], Predicciones - [3.8154347 3.3711965 3.05769   ... 2.0102482 3.1489937 2.8433867]
Usuario 2: Películas no vistas - [   0    1    2 ... 1679 1680 1681], Predicciones - [3.7859993 3.323765  2.9882278 ... 1.9850111 3.1108987 2.8074074]
Usuario 3: Películas no vistas - [   0    1    2 ... 1679 1680 1681], Predicciones - [3.6103837 3.1303995 2.7656462 ... 1.8820708 2.9510472 2.6603885]
Usuario 4: Películas no vistas - [   0    1    2 ... 1679 1680 1681], Predicciones - [4.8062277 4.634559  4.6943264 ... 2.686723  4.1885853 3.8099391]
Usuario 5: Películas no vistas - [   2    3    4 ... 1679 1680 1681], Predicciones - [2.5310466 3.0144536 2.796514  ... 1.7682858 2.7738578 2.498066 ]
Usuario 6: Películas no vistas - [   0    1    2 ... 1679 1680 1681], Predicciones - [3.3460698 2.8287356 2.4093277 ... 1.7213618 2.7024477 2.4311678]
Usuario 7: Películas no vistas - [   0    1    2 ... 1679 1680 1681], Predicciones - [4.227103